In [ ]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

In [4]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [5]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [6]:
df.head()

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event


### Step 1 : Create source embeddings for the text column


In [7]:
from sentence_transformers import SentenceTransformer


In [8]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
vectors.shape


(8, 768)

In [11]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors


In [12]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [13]:
index.add(vectors)


In [14]:
index


<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x2877e8f60> >

### Step 4 : Encode search text using same encorder and normalize the output vector


In [33]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [34]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

### Step 5: Search for similar vector in the FAISS index created


In [35]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844838, 1.4039094]], dtype=float32)

In [36]:
I

array([[3, 2]])

In [37]:
I.tolist()

[[3, 2]]

In [38]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [39]:
df.loc[row_indices]


,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [40]:
search_query


'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query